In [ ]:
!pip install hatesonar

In [1]:
import json

In [2]:
import pandas as pd
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from LoadingData import LoadingData
import numpy as np

import nltk
nltk.download('punkt')
from nltk import word_tokenize

from hatesonar import Sonar


[nltk_data] Downloading package punkt to /Users/nissani/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
def clean_getty_data():
    data = LoadingData.LoadingData("~/Hateful_Memes_Project/data/getty_data_full.csv")
    getty=data.load_data()
    df=getty[~getty['image'].str.contains("\(")]
    getty[~getty['image'].str.contains("\(")].image.nunique()
    
    df['color_zscore'] = (df['Color_Score'] - df['Color_Score'].mean())/df['Color_Score'].std(ddof=0)
    df['key_point_zscore'] = (df['Key_Point_Score'] - df['Key_Point_Score'].mean())/df['Key_Point_Score'].std(ddof=0)
    df['combined_z']=df['color_zscore']+df['key_point_zscore']
    df['combined_zscore']=(df['combined_z'] - df['combined_z'].mean())/df['combined_z'].std(ddof=0)
    
#     s = df.combined_zscore
#     ax = s.plot.kde()
#     display(df.combined_zscore.describe())
    
    combined_index = df.groupby('image')['combined_zscore'].idxmin()
    match=df.loc[combined_index]
    
    return match

In [4]:
def load_full_dataset():
    df_list = []

    for x in ['train','dev','test']:
        file="../data/"+x+".jsonl"
        data=LoadingData.LoadingData(file)
        data_json=data.load_data()
        df=pd.DataFrame(data_json)
        df['partition']=x
        if x=='test':
            df['label']=np.nan
        df_list.append(df)
    
    final = pd.concat(df_list)
    return final
    
    

In [5]:
def create_meme_text_hate_speech_features(clean):
    sonar = Sonar()
    clean['hate_speech']=clean.text.apply(lambda x: sonar.ping(text=str(x or '')).get('classes')[0].get('confidence'))
    clean['offensive_langauge']=clean.text.apply(lambda x: sonar.ping(text=str(x or '')).get('classes')[1].get('confidence'))
    clean['neither']=clean.text.apply(lambda x: sonar.ping(text=str(x or '')).get('classes')[2].get('confidence'))

In [6]:
def create_simple_getty_features():
    #load data.  Match is best matching Getty data and df is full dataframe of train/test/dev data
    match=clean_getty_data()
    df=load_full_dataset()
    
    
    #clean up types, column names, and text
    match['src']=match['src'].astype(str)
    df['id']=df['id'].astype(str)
    match.rename(columns={'id':'rank'}, inplace=True)
    
    full=df.merge(match, how='left', left_on='id', right_on='src')
    full['tags_clean']= full['tags'].str.replace(',',' ').str.replace('Photos',' ').str.lower().str.split()
    
    #tokenize text
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    full['text_clean'] = full['text'].apply(lambda x: tokenizer.tokenize(x))
    
    
    clean=full[['id','img','text','text_clean','partition','tags','tags_clean','caption','combined_zscore','label']]
    
    #calculate text frequencies
    clean['all_text_freq']= clean.text.map(clean.text.value_counts())
    clean['train_text_freq']= clean.text.map(clean[clean.partition=='train'].text.value_counts())
    clean['dev_text_freq']= clean.text.map(clean[clean.partition=='dev'].text.value_counts())
    clean['test_text_freq']= clean.text.map(clean[clean.partition=='test'].text.value_counts())

    # manual lists for features.  This snowballed into something ugly, refactor in a better way, maybe dict.
    race=['african','black','asian','chinese','arab','white','african-american','ethnicity']
    disability=['disabled','disability',"down's, syndrome",'retarded','retarted']
    religion=['islam','muslim','muslims','catholic','catholics','christian','christians','jewish','jew','jews',
             'god','jesus']
    sexual_orientation=['gay','straight','trans','transexual','homo','homosexual']
    violence=['obscene','anger','aggression','kill','killing','bomb','gun']

    criminals=['hitler']
    animals=['alligator','ant','bear','bee','bird','camel','cat','cheetah','chicken','chimpanzee','cow','crocodile','deer','dog','dolphin','duck','eagle','elephant','fish','fly','fox','frog','giraffe','goat','goldfish','hamster','hippopotamus','horse','kangaroo','kitten','lion','lobster','monkey','octopus','owl','panda','pig','puppy','rabbit','rat','scorpion','seal','shark','sheep','snail','snake','spider','squirrel','tiger','turtle','wolf','zebra']

    gender=['man','woman','men','women','bitch','pussy']

    #simple features for baseline
    clean['tags_clean'] = [ [] if x is np.NaN else x for x in clean['tags_clean'] ]
    clean['tags_race'] = clean.apply(lambda row: True if any(item in race for item in row['tags_clean']) else False, axis = 1)
    clean['tags_disability'] = clean.apply(lambda row: True if any(item in disability for item in row['tags_clean']) else False, axis = 1)
    clean['tags_religion'] = clean.apply(lambda row: True if any(item in religion for item in row['tags_clean']) else False, axis = 1)
    clean['tags_sexual_orientation'] = clean.apply(lambda row: True if any(item in sexual_orientation for item in row['tags_clean']) else False, axis = 1)
    clean['tags_violence'] = clean.apply(lambda row: True if any(item in violence for item in row['tags_clean']) else False, axis = 1)
    clean['tags_criminals'] = clean.apply(lambda row: True if any(item in criminals for item in row['tags_clean']) else False, axis = 1)
    clean['tags_gender'] = clean.apply(lambda row: True if any(item in gender for item in row['tags_clean']) else False, axis = 1)
    clean['tags_animals'] = clean.apply(lambda row: True if any(item in animals for item in row['tags_clean']) else False, axis = 1)

    clean['text_clean'] = [ [] if x is np.NaN else x for x in clean['text_clean'] ]
    clean['text_race'] = clean.apply(lambda row: True if any(item in race for item in row['text_clean']) else False, axis = 1)
    clean['text_disability'] = clean.apply(lambda row: True if any(item in disability for item in row['text_clean']) else False, axis = 1)
    clean['text_religion'] = clean.apply(lambda row: True if any(item in religion for item in row['text_clean']) else False, axis = 1)
    clean['text_sexual_orientation'] = clean.apply(lambda row: True if any(item in sexual_orientation for item in row['text_clean']) else False, axis = 1)
    clean['text_violence'] = clean.apply(lambda row: True if any(item in violence for item in row['text_clean']) else False, axis = 1)
    clean['text_criminals'] = clean.apply(lambda row: True if any(item in criminals for item in row['text_clean']) else False, axis = 1)
    clean['text_gender'] = clean.apply(lambda row: True if any(item in gender for item in row['text_clean']) else False, axis = 1)
    clean['text_animals'] = clean.apply(lambda row: True if any(item in animals for item in row['text_clean']) else False, axis = 1)
    
    #Add hate speech features
    create_meme_text_hate_speech_features(clean)
    
    return clean

In [8]:
clean=create_simple_getty_features()

FileNotFoundError: [Errno 2] File /Users/nissani/Hateful_Memes_Project/data/getty_data_full.csv does not exist: '/Users/nissani/Hateful_Memes_Project/data/getty_data_full.csv'

In [ ]:
clean.to_csv( "../data/getty_features.csv", index=False)

### Simple Decision Tree Model

In [ ]:
clean.head(1)

In [ ]:
clean.columns

In [ ]:
clean.combined_zscore.fillna(2, inplace=True)
# clean.all_text_freq.fillna(0, inplace=True)
# clean.train_text_freq.fillna(0, inplace=True)
# clean.dev_text_freq.fillna(0, inplace=True)

In [ ]:
features=clean.columns[14:].tolist()
# features.append('combined_zscore')
# features=clean.columns[13:].tolist()
# cols.append('id')
# cols.append('label')
features

In [ ]:
X = clean[clean.partition=='train'][features]
y = clean[clean.partition=='train'].label

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1) # 70% training and 30% test

In [ ]:
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

In [ ]:
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

In [ ]:
from sklearn.metrics import roc_auc_score
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("AUC:",roc_auc_score(y_test, y_pred))

In [ ]:
y_test.value_counts()

In [ ]:
from sklearn.metrics import confusion_matrix

c=confusion_matrix(y_test, y_pred)
c / c.astype(np.float).sum(axis=1)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
feature_imp = pd.Series(clf.feature_importances_,index=features).sort_values(ascending=False)
feature_imp

%matplotlib inline
# Creating a bar plot
sns.barplot(x=feature_imp, y=feature_imp.index)
# Add labels to your graph
plt.xlabel('Feature Importance Score')
plt.ylabel('Features')
plt.title("Visualizing Important Features")
plt.legend()
plt.show()

In [ ]:
X_train.head()

### Nice Illustration of "comparison to animal" hate speech category

In [ ]:
clean[clean.text=='sea monkeys'].head()

In [ ]:
### features needed:
# image_has_animal
# image_has_person
# image_has_criminal
# image_has_race
# image_has_religion

In [ ]:
clean[~clean.test_text_freq.isna()].all_text_freq.value_counts()

In [ ]:
clean[(clean.test_text_freq==clean.all_text_freq)&(~clean.test_text_freq.isna())].head()

In [ ]:
clean[clean.text=="doesn't wear tampons because she trips over the string"]

In [ ]:
clean[(clean.test_text_freq!=clean.all_text_freq)&(~clean.test_text_freq.isna())].head()